# 휴먼지능정보공학과 201810808 정민지

문제: zscore, cdf 계산



성적데이터는 n이 적지만, 정규분포를 이룬다고 가정하자.

* 1-1 성적데이터로 DataFrame을 생성.

* 1-2 zscore 컬럼을 생성.

zscore를 계산하려면, 평균과 표준편차를 알아야 한다.

계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.

* 1-3 cdf 컬럼을 생성.

scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다.

cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

---

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

## 1-1 성적데이터로 DataFrame을 생성.

In [2]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

#### RDD 생성

In [3]:
_marksRdd=spark.sparkContext.parallelize(marks)

In [4]:
_marksRdd.take(1)

['김하나, English, 100']

#### Dataframe 생성

In [5]:
from pyspark.sql import Row

_csv = _marksRdd.map(lambda l: l.split(",")) #map() 함수를 사용하여 csv 형식으로 구성된 파일을 컴마(,)로 분리
_csvRdd = _csv.map(lambda p: 
    Row(
        name=p[0], 
        subject=p[1],
        mark=float(p[2])
    )
)

### > 답

In [6]:
#RDD를 Dataframe으로 변환한다.
_df=spark.createDataFrame(_csvRdd)

In [7]:
_df.printSchema()
_df.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: double (nullable = true)

+------+--------+-----+
|  name| subject| mark|
+------+--------+-----+
|김하나| English|100.0|
|김하나|    Math| 80.0|
|임하나| English| 70.0|
|임하나|    Math|100.0|
|김갑돌| English| 82.3|
|김갑돌|    Math| 98.5|
+------+--------+-----+



---
## 1-2 zscore 컬럼을 생성

#### _df를 rdd로 변환하여 mark 배열 추출

In [8]:
_mark=_df.rdd.map(lambda x:x[2]).collect()

In [9]:
_mark

[100.0, 80.0, 70.0, 100.0, 82.3, 98.5]

#### zscore 위한 변수 선언

In [10]:
import numpy as np

xbar=np.mean(_mark)
sigmax=np.std(_mark) #ddof=0
sx=np.std(_mark,ddof=1)

#### _df에 zscore컬럼 추가

In [11]:
_df=_df.withColumn('zscore',(_df['mark']-xbar)/sx)

### > 답

In [12]:
_df.show()

+------+--------+-----+-------------------+
|  name| subject| mark|             zscore|
+------+--------+-----+-------------------+
|김하나| English|100.0| 0.9020148231647499|
|김하나|    Math| 80.0| -0.662172731456202|
|임하나| English| 70.0| -1.444266508766678|
|임하나|    Math|100.0| 0.9020148231647499|
|김갑돌| English| 82.3|-0.4822911626747928|
|김갑돌|    Math| 98.5| 0.7847007565681785|
+------+--------+-----+-------------------+



In [14]:
_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: double (nullable = true)
 |-- zscore: double (nullable = true)



---
## 1-3 cdf 컬럼을 생성

In [87]:
from scipy.stats import norm
from pyspark.sql.functions import udf

In [88]:
cdf_udf=udf(lambda x: float(norm.cdf(x)))

In [89]:
dffinal=_df.withColumn("cdf", cdf_udf(_df['zscore']))

In [90]:
dffinal.show()

+------+--------+-----+-------------------+-------------------+
|  name| subject| mark|             zscore|                cdf|
+------+--------+-----+-------------------+-------------------+
|김하나| English|100.0| 0.9020148231647499| 0.8164755032306112|
|김하나|    Math| 80.0| -0.662172731456202| 0.2539302639589459|
|임하나| English| 70.0| -1.444266508766678|0.07433201139097219|
|임하나|    Math|100.0| 0.9020148231647499| 0.8164755032306112|
|김갑돌| English| 82.3|-0.4822911626747928|0.31479956212721416|
|김갑돌|    Math| 98.5| 0.7847007565681785| 0.7836854804484269|
+------+--------+-----+-------------------+-------------------+



---
---
---
---

# try

In [ ]:
#from pyspark.sql.functions import udf 안하고 F만 불러와도 걍 되는듯? sql function아래 있어서 그런가

In [67]:
from scipy.stats import norm
from pyspark.sql import functions as F
norm_cdf = F.udf(lambda x: float(norm.cdf(x)))



In [73]:
dff=_df.withColumn("cdf", norm_cdf(_df['zscore']))

In [74]:
dff.show()

+------+--------+-----+-------------------+-------------------+
|  name| subject| mark|             zscore|                cdf|
+------+--------+-----+-------------------+-------------------+
|김하나| English|100.0| 0.9020148231647499| 0.8164755032306112|
|김하나|    Math| 80.0| -0.662172731456202| 0.2539302639589459|
|임하나| English| 70.0| -1.444266508766678|0.07433201139097219|
|임하나|    Math|100.0| 0.9020148231647499| 0.8164755032306112|
|김갑돌| English| 82.3|-0.4822911626747928|0.31479956212721416|
|김갑돌|    Math| 98.5| 0.7847007565681785| 0.7836854804484269|
+------+--------+-----+-------------------+-------------------+



---

In [91]:
# 솔직히 똑같이 했는데 이 오류 왜뜨는지 모르겠음..

In [72]:
_dddd=_df.withColumn('s',float(norm.cdf(_df['zscore'])))

TypeError: iteration over a 0-d array

---

In [ ]:
# 배열을 df로 (사실 합칠수없어서 쓸데없을지도..) pandas에서 함침->다시 돌림 이러면 될듯?

In [59]:
arr=_mark

In [60]:
from pyspark.sql.types import FloatType
df_zscore=spark.createDataFrame(arr,FloatType()) #schema를 선언해줘야함

In [61]:
df_zscore.show()

+-----+
|value|
+-----+
|100.0|
| 80.0|
| 70.0|
|100.0|
| 82.3|
| 98.5|
+-----+



---

In [ ]:
# spark df -> pandas df

In [ ]:
판다스df.toPandas()

In [9]:
#예시
pdf=_df.toPandas()
pdf

,name,subject,mark
0,김하나,English,100.0
1,김하나,Math,80.0
2,임하나,English,70.0
3,임하나,Math,100.0
4,김갑돌,English,82.3
5,김갑돌,Math,98.5


---

In [ ]:
# pandas df -> spark df

In [ ]:
spark.createDataFrame(판다스df)

In [11]:
#예시
sdf=spark.createDataFrame(pdf)
sdf.show()

+------+--------+-----+
|  name| subject| mark|
+------+--------+-----+
|김하나| English|100.0|
|김하나|    Math| 80.0|
|임하나| English| 70.0|
|임하나|    Math|100.0|
|김갑돌| English| 82.3|
|김갑돌|    Math| 98.5|
+------+--------+-----+

